In [1]:
import numpy as np
import pandas as pd
#import seaborn as sns
from glob import glob
from PIL import Image
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from imutils import paths
import random

from subprocess import check_output

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.applications.densenet import preprocess_input
from keras.preprocessing.image import (random_rotation, random_shift, random_shear, random_zoom)


C:\Users\jdu12\Anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# load a single image to np array
def get_image(img_path):
    img = load_img(img_path, target_size=(224, 224))#.convert('L')
    img = img_to_array(img)
    img = img/255
    return img

def to_one_hot(labels):
    id2class = dict()
    class2id = dict()      ##!!! class(integer) to id(whale class)
    num_labels = []
    counter = 0
    for id in labels:
        if id not in id2class.keys():
            num_labels.append(counter)
            id2class[id] = counter
            class2id[counter] = id
            counter += 1
        else:
            num_labels.append(id2class[id])
    one_hot_labels = to_categorical(num_labels, num_classes = 4251)
    return one_hot_labels, class2id, id2class, num_labels


train_label_dir = 'C:/Users/jdu12/Desktop/humpback/train.csv'
df = pd.read_csv(train_label_dir)

labels = df['Id']
labels, class2id, id2class, num_labels = to_one_hot(labels)

# generate class weights
from sklearn.utils import class_weight
cw = class_weight.compute_class_weight('balanced', np.arange(len(class2id)), np.asarray(num_labels))
cw = dict(enumerate(cw.flatten(), 1))
# since cw start at 1 instead of 0, we need to offset this by 1
cw_z = dict()
for key in cw:
    cw_z[key-1] = cw[key]

In [3]:
# loading train images
trainPaths = list(paths.list_images('C:/Users/jdu12/Desktop/humpback/train/'))
print(len(trainPaths))
train = np.array([get_image(img_path) for img_path in trainPaths])
print(train.shape)

9082
(9082, 224, 224, 3)


In [ ]:
# split the data into train and val
# The dataset is super unbalanced, as there are many classes that contains only 1 image
# As a result, train/val data cannot be split before generating more data by augmentation
train_dir = 'C:/Users/jdu12/Desktop/humpback/train/'

num_categories = len(df['Id'].unique())
#print(num_categories)
validation = np.zeros((num_categories, 224, 224, 3))
validation_y = []

i = 0
for id in df['Id'].unique():
    validation_y.append(id)
    im = df[df['Id'] == id].sample(1)
    name =  np.array(im.get('Image'))[0]
    im = get_image(train_dir + name)

    # https://www.kaggle.com/lextoumbourou/humpback-whale-id-data-and-aug-exploration
    x = random.randint(0, 3)
    if x == 0:
        validation[i,:, :, :] = random_rotation(im, 30, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    if x == 1:
        validation[i,:, :, :] = random_shift(im, wrg=0.1, hrg=0.3, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    if x == 2:
        validation[i,:, :, :] = random_shear(im, intensity=0.4, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    if x == 3:
        validation[i,:, :, :] = random_zoom(im, zoom_range=(1.5, 0.7), row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    i = i + 1

validation_y = to_one_hot(validation_y)[0]

In [3]:
# define data generator
#use of an image generator for preprocessing and data augmentation
image_gen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    #rescale=1./255,
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True
)

In [6]:
# acquire the model
model = keras.applications.densenet.DenseNet121(include_top=True, weights=None, classes=4251)

In [7]:
batch_size = 32
epochs = 200

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_dir = 'C:/Users/jdu12/Desktop/humpback/saved_model/DenseNet/'
routine_dir = model_dir + "routine-{epoch:02d}-{acc:.2f}.hdf5"
routine_save = ModelCheckpoint(routine_dir, monitor='acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=10)
best_dir = model_dir + "best-{epoch:02d}-{acc:.2f}.hdf5"
best_save = ModelCheckpoint(best_dir, monitor='acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=4)


hist = model.fit_generator(image_gen.flow(train, labels, batch_size=batch_size),
                    steps_per_epoch=train.shape[0]//batch_size,
                    epochs=epochs, verbose=1, callbacks=[routine_save, best_save], class_weight=cw_z,
                    validation_data=(validation, validation_y))


Epoch 1/200
283/283 [==============================] - 137s 484ms/step - loss: 8.4349 - acc: 4.6709e-04 - val_loss: 8.9887 - val_acc: 9.4096e-04
Epoch 2/200
283/283 [==============================] - 119s 421ms/step - loss: 8.1795 - acc: 3.3127e-04 - val_loss: 9.6110 - val_acc: 0.0012
Epoch 3/200
283/283 [==============================] - 119s 420ms/step - loss: 7.7535 - acc: 9.9382e-04 - val_loss: 7.5623 - val_acc: 0.0049
Epoch 4/200
283/283 [==============================] - 118s 417ms/step - loss: 7.2089 - acc: 0.0014 - val_loss: 7.0625 - val_acc: 0.0113
Epoch 5/200
283/283 [==============================] - 134s 473ms/step - loss: 6.7679 - acc: 0.0029 - val_loss: 8.6470 - val_acc: 0.0042
Epoch 6/200
283/283 [==============================] - 134s 473ms/step - loss: 6.3023 - acc: 0.0052 - val_loss: 6.1017 - val_acc: 0.0315
Epoch 7/200
283/283 [==============================] - 133s 470ms/step - loss: 5.7972 - acc: 0.0127 - val_loss: 6.0787 - val_acc: 0.0553
Epoch 8/200
283/283 [====

283/283 [==============================] - 132s 465ms/step - loss: 0.0996 - acc: 0.8715 - val_loss: 0.2388 - val_acc: 0.9412
Epoch 61/200
283/283 [==============================] - 132s 465ms/step - loss: 0.0909 - acc: 0.8776 - val_loss: 0.7910 - val_acc: 0.8172
Epoch 62/200
283/283 [==============================] - 131s 463ms/step - loss: 0.0796 - acc: 0.8788 - val_loss: 0.3054 - val_acc: 0.9280
Epoch 63/200
283/283 [==============================] - 130s 461ms/step - loss: 0.0975 - acc: 0.8763 - val_loss: 0.3345 - val_acc: 0.9188
Epoch 64/200
283/283 [==============================] - 132s 465ms/step - loss: 0.1283 - acc: 0.8632 - val_loss: 0.5022 - val_acc: 0.8829
Epoch 65/200
283/283 [==============================] - 131s 461ms/step - loss: 0.1160 - acc: 0.8662 - val_loss: 0.3185 - val_acc: 0.9278
Epoch 66/200
283/283 [==============================] - 130s 459ms/step - loss: 0.0839 - acc: 0.8815 - val_loss: 0.4894 - val_acc: 0.8817
Epoch 67/200
283/283 [=========================

283/283 [==============================] - 128s 451ms/step - loss: 0.0206 - acc: 0.9203 - val_loss: 0.1205 - val_acc: 0.9739
Epoch 120/200
283/283 [==============================] - 128s 452ms/step - loss: 0.0321 - acc: 0.9206 - val_loss: 0.3331 - val_acc: 0.9344
Epoch 121/200
283/283 [==============================] - 129s 455ms/step - loss: 0.0315 - acc: 0.9124 - val_loss: 0.0938 - val_acc: 0.9777
Epoch 122/200
283/283 [==============================] - 130s 459ms/step - loss: 0.0541 - acc: 0.9115 - val_loss: 0.3732 - val_acc: 0.9134
Epoch 123/200
283/283 [==============================] - 130s 458ms/step - loss: 0.0518 - acc: 0.8980 - val_loss: 0.1669 - val_acc: 0.9600
Epoch 124/200
283/283 [==============================] - 130s 460ms/step - loss: 0.0397 - acc: 0.9125 - val_loss: 0.6053 - val_acc: 0.9134
Epoch 125/200
283/283 [==============================] - 129s 456ms/step - loss: 0.0457 - acc: 0.9120 - val_loss: 0.4925 - val_acc: 0.9306
Epoch 126/200
283/283 [==================

283/283 [==============================] - 127s 447ms/step - loss: 0.0270 - acc: 0.9260 - val_loss: 0.0781 - val_acc: 0.9791
Epoch 179/200
283/283 [==============================] - 127s 448ms/step - loss: 0.0116 - acc: 0.9392 - val_loss: 0.0665 - val_acc: 0.9831
Epoch 180/200
283/283 [==============================] - 128s 451ms/step - loss: 0.0109 - acc: 0.9430 - val_loss: 0.0525 - val_acc: 0.9871
Epoch 181/200
283/283 [==============================] - 127s 448ms/step - loss: 0.0119 - acc: 0.9456 - val_loss: 0.0540 - val_acc: 0.9880
Epoch 182/200
283/283 [==============================] - 127s 448ms/step - loss: 0.0105 - acc: 0.9465 - val_loss: 0.0778 - val_acc: 0.9793
Epoch 183/200
283/283 [==============================] - 127s 449ms/step - loss: 0.0329 - acc: 0.9237 - val_loss: 0.5732 - val_acc: 0.8901
Epoch 184/200
283/283 [==============================] - 126s 447ms/step - loss: 0.0283 - acc: 0.9142 - val_loss: 0.4425 - val_acc: 0.9337
Epoch 185/200
283/283 [==================

In [5]:
# loading test images
testPaths = list(paths.list_images('C:/Users/jdu12/Desktop/humpback/test/'))
print(len(testPaths))
test_images = np.array([get_image(img_path) for img_path in testPaths])
print(test_images.shape)

15610
(15610, 224, 224, 3)


In [6]:
# loading trained model
model_name = 'routine-200-0.94'
model = load_model('C:/Users/jdu12/Desktop/humpback/saved_model/DenseNet/' + model_name+ '.hdf5')

In [7]:
# predict!
pred = model.predict(test_images)

# testing and generating submission file
import warnings
from os.path import split

pred_dir = "C:/Users/jdu12/Desktop/humpback/prediction/DenseNet/"

with open(pred_dir + model_name + ".csv", "w") as f:
    f.write("Image,Id\n")
    top_5 = np.argsort(pred)[:,-1:-6:-1]   # get the top 5 most likely classes
    for i in range(top_5.shape[0]):
        cur_tags = ''
        cur_image_name = testPaths[i].split('/')[-1]
        for j in range(5):
            cur_tags = cur_tags + ' ' + class2id[top_5[i][j]]
        f.write("%s,%s\n" %(cur_image_name, cur_tags))     